In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import holoviews as hv
hv.extension('bokeh')
import bokeh
from bokeh.io import output_notebook, show
import bokeh.plotting as bp # figure, show, output_file
from talk_plottingutils import *
output_notebook()

import functools
import toolz

# Cell fate prediction in hematopoietic stem cells
[Prospective identification of hematopoietic lineage choice by deep learning](http://www.nature.com/nmeth/journal/v14/n4/full/nmeth.4182.html)    
Felix Buggenthin, Florian Buettner, Philipp S Hoppe, Max Endele, Manuel Kroiss, Michael Strasser, Michael Schwarzfischer, Dirk Loeffler, Konstantinos D Kokkaliaris, Oliver Hilsenbeck, Timm Schroeder, Fabian J Theis, Carsten Marr<p>
Nature Methods 14, 403–406 (2017)


[Code](https://github.com/QSCD/HematoFatePrediction) 

## Quick recap:
- murine blood stem cells differentiate into distinct cell types


<!--![Orkin1](orkin1.png)-->
![Orkin2](images/orkin2.png)
<img src="images/time_lapse_setup.png" alt="time_lapse" style="width: 200px;"/>

- continuous in vitro imaging of this process over several days

# Raw data
brightfield images

<img src="images/movie_positions.png" alt="Position layout" style="width: 400px;"/>
<img src="images/single_position.png" alt="single position" style="width: 400px;"/>

# Handling the data
[movieTools](https://github.com/redst4r/movieTools.git) is a convenient wrapper around the time-lapse data and should already be installed if you followed these [instructions](0_overview.ipynb).

In [ ]:
import movieTools.config
import movieTools.imageNormalizer as imageNormalizer
from movieTools.movie import Movie
movieTools.config.TTTDIR = '/home/michi/pythonProjects/deepLearning/Hemato_korea/data_small/'  # change that to your location!

%ls /home/michi/pythonProjects/deepLearning/Hemato_korea/data_small/
print("\n---------------\n")
%ls /home/michi/pythonProjects/deepLearning/Hemato_korea/data_small/experiment3/

In [ ]:
movie = Movie('experiment3', verbose=False)  # 140206

print('Positions')
print(movie.get_all_positions())

print('\nTimepoints in position 54')
print(len(movie.get_timepoints(position=54)))

print('\nTotal number of images')
len(movie.get_all_images())  # misnomer here: it returns just a dict of timepoint/position -> filename



Lets load a single raw image from position 54, timepoint 3330

**Note**: The downloaded data contains only timepoints 3300-3399 (usually, 1-5000)

In [ ]:
I = movie.loadimage(position=54, 
                    timepoint=3330, 
                    WL='w00',        # Brightfield
                    extension='png', # for backwards compatibility
                    normalizer=imageNormalizer.NoNormalizer())  # just load raw images, no background correction
plot_image_bokeh(I);

Load all images between t=3330 and t=3350

In [ ]:
def load_raw_image(position, timepoint):
    "just to get rid of some of the arguments"
    return movie.loadimage(position, timepoint, WL='w00', extension='png', normalizer=imageNormalizer.NoNormalizer())

# For performance reasons (browser related), just save a smaller part of the image at half resolution
tt = range(3330,3350)
img_list = [load_raw_image(position=54, timepoint=i)[:600:2,:600:2]
            for i in tt]

In [ ]:
hv_original_stack = hv_plot_stack(img_list, tt)

In [ ]:
%%output size=150
%%opts Image style(cmap='Greys_r')
hv_original_stack 

# Observations
- uneven illumination, strange flickering
- borders of the cover-slip
- dirt

Lets have a look at this in more detail.

**Task 1:**
What's the average image intensity over time, i.e. $E_x[I(t,x)]$?

In [ ]:
average_intensity_over_time = ...

**Solution 1**

In [ ]:
%load solutions/bg-01.py

plot it

In [ ]:
plt.plot(average_intensity_over_time)
plt.xlabel('Time')
plt.ylabel('avg. intensity')
plt.show()

**Task 2:** What's the average image $E_t[I(t,x)]$

In [ ]:
meanI = ...

**Solution 2**

In [ ]:
%load solutions/bg-02.py

plot it

In [ ]:
plot_image_bokeh(meanI);  # plot_image_mpl(meanI); plt.grid(False); plt.show()

# Background normalization
How do we get rid of these experimental artifacts in the images?

**Image equation**:

$$I(x,t) = [B(t) \cdot I^{true}(x,t)] \cdot S(x) + D(x)$$
- $S(x)$ flatfield, uneven illumination
- $D(x)$ darkfield, camera offset, noise
- $B(t)$ time dependent baseline intensity (i.e. due to photobleaching)

**Methods**

1. neglect darkfield, estimate $S$, $B$ from the raw data

3. more sophisticated: [BaSiC](https://github.com/QSCD/BaSiC). Takes care of all at once, enforcing some more constraints


## simple normalization:
Assume that $S(x)$ is the time average, $B(t)$ is the space average as estimated above

**Task 3**: apply this simple normalization to pos=54, time = 3310

In [ ]:
Iobs =... #original image
Inorm = ...  # normalized

**Solution 3**

In [ ]:
%load solutions/bg-03.py

In [ ]:
plot_image_bokeh(Inorm);

## BaSiC

    A BaSiC Tool for Background and Shading Correction of Optical Microscopy Images
    Tingying Peng, Kurt Thorn, Timm Schroeder, Lichao Wang, Fabian J Theis, Carsten Marr, Nassir Navab
    Nature Communication 8:14836 (2017)
    
    
    
$$I(x,t) = [B(t) \cdot I^{true}(x,t)] \cdot S(x) + D(x)$$
- $S(x)$ flatfield, uneven illumination
- $D(x)$ darkfield, camera offset, noise
- $B(t)$ time dependent baseline intensity (i.e. due to photobleaching)
- smoothness constrains on $S(x),D(x)$ (sparse in Fourier space)

In [ ]:
I0 = movie.loadimage(position=54, timepoint=3310, WL='w00', extension='png', 
                     normalizer= imageNormalizer.NoNormalizer())

I_basic = movie.loadimage(position=54, timepoint=3310, WL='w00', extension='png', 
                          normalizer= imageNormalizer.BaSiC_Normalizer())

plt.figure(figsize=(20,20))
plt.subplot(1,2,1)
plt.imshow(I0, cmap=plt.cm.Greys_r)
plt.subplot(1,2,2)
plt.imshow(I_basic, cmap=plt.cm.Greys_r)
plt.show()

# Compare to the original frames at the top of the notebook

** Normalize the entire stack of images from t=3430-3450**

In [ ]:
# a couple of consecutive frames
tt = range(3330,3350)
img_list_corrected = [movie.loadimage(position=54, timepoint=i, WL='w00', extension='png', 
                                      normalizer=imageNormalizer.BaSiC_Normalizer())[:600:2,:600:2]
        for i in tt]

In [ ]:
hv_corrected_stack = hv_plot_stack(img_list_corrected, tt)

In [ ]:
%%output size=100
%%opts Image style(cmap='Greys_r')
hv_original_stack + hv_corrected_stack

# A larger timeframe

In [ ]:
tt = range(3300,3400, 2)
img_list = [movie.loadimage(position=54, timepoint=i, WL='w00', extension='png', 
                        normalizer=imageNormalizer.BaSiC_Normalizer())[:600:2,:600:2]
        for i in tt]

In [ ]:
%%output size=150
%%opts Image style(cmap='Greys_r')
hv_plot_stack(img_list, tt)